In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/Users/sanghyuk/myProjects/mlproject02'

In [4]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: List[str]

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from pathlib import Path

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        
        create_directories([Path(config.root_dir)])
        
        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )
        
        return data_validation_config

In [8]:
import os
from textSummarizer.logging import logger

In [11]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None
            
            all_files = os.listdir(os.path.join('artifacts', 'data_ingestion', 'samsum_dataset'))
            
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}, missing file: {file}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}, all files are present")
                        
            return validation_status
        
        except Exception as e:
            logger.error(f"Error occurred while validating files: {e}")
            return False
            


In [12]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    logger.exception(e)
    raise e

[2024-07-10 11:47:40,102: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-10 11:47:40,104: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-10 11:47:40,105: INFO: common: Created directory at: artifacts]
[2024-07-10 11:47:40,106: INFO: common: Created directory at: artifacts/data_validation]
